In [2]:
from SimPEG import *
from simpegEM1D import (
    Utils1D, get_vertical_discretization_time, 
    set_mesh_1d, skytem_HM_2015
)
from simpegEM1D.simulation_original import EM1DTMSimulation
from simpegEM1D.survey_original import EM1DSurveyTD
from simpegEM1D import analytics
import numpy as np
from simpegEM1D import skytem_HM_2015
from simpegEM1D.supporting_functions.digital_filter import setFrequency, transFilt
from scipy.constants import mu_0
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
time = np.logspace(-5, -2, 31)
# three layers
hz = np.array([10, 10, 10])
sigma_half = 0.1

# half-space conductivity
sigma = np.ones(hz.size) * sigma_half
mesh1D = set_mesh_1d(hz)
depth = -mesh1D.gridN[:-1]
# offset b/w VMD source and Bz receiver
offset = np.array([10.])

LocSigZ = -mesh1D.gridCC
TDsurvey = EM1DSurveyTD(
    rx_location = np.array([0., 0., 100.]),
    src_location = np.array([0., 0., 100.]),
    topo = np.r_[0., 0., 100.],
    depth = depth,
    rx_type = 'Bz',
    wave_type = 'stepoff',
    src_type = 'VMD',
    offset=offset,
    time = time,
)
imap = maps.IdentityMap(mesh1D)
sim = EM1DTMSimulation(mesh1D, survey=TDsurvey, sigmaMap=imap, verbose=False)
bz = sim.dpred(sigma)
print(np.shape(bz))
print(bz)

AttributeError: 'EM1DTMSimulation' object has no attribute 'n_layer'

In [4]:
def bz_analytic_solution(r, t, sigma):
    """
    r: offset bewteen VMD soure and Bz receiver (m)
    t: measured times
    sigma: conductivity of the half-space earth
    """
    wt, tbase, omega_int = setFrequency(t)
    hz = analytics.Hzanal(sigma, omega_int/2/np.pi, r, 'secondary')
    # Treatment for inaccuracy in analytic solutions
    ind = omega_int < 0.2
    hz[ind] = 0.
    hzTD, f0 = transFilt(hz, wt, tbase, omega_int, t)
    return hzTD*mu_0

In [ ]:
bz_analytic = bz_analytic_solution(offset, time, sigma_half)
print(np.shape(bz_analytic))
print(bz_analytic)

In [ ]:
fig, axes = subplots(1,1, figsize = (7,5))
axes.loglog(TDsurvey.time, bz_analytic, '-')
axes.loglog(TDsurvey.time, bz, 'o')
axes.set_xscale('log')
axes.set_yscale('log')
axes.set_xlabel('Time (s)')
axes.set_ylabel('Bz (T)')
axes.legend(("Analytic", "simpegEM1D"))